In [130]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
import torch
import torchtext
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
import random
from typing import Tuple
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor
import math
import time

In [131]:
main_df = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/train.csv')


In [132]:
test_df = pd.read_csv('/kaggle/input/feedback-prize-english-language-learning/test.csv')
test_df['cohesion'] = 1.0
test_df['syntax'] = 1.0
test_df['vocabulary'] = 1.0 
test_df['phraseology'] = 1.0 
test_df['grammar'] = 1.0
test_df['conventions'] = 1.0 

In [133]:
train_df, val_df = train_test_split(main_df, test_size=0.03, random_state=6)

In [134]:
tokenizer = get_tokenizer('spacy', language='en')
MAX_SENT_LEN = 1000

def mark_bos_eos(text):
    final_text = ''
    final_text = final_text.strip()
    l = text.split('.')[:-1]
    for sentence in l:
        final_text = final_text + ' <bos> ' + sentence + ' <eos> '
    final_text = final_text.replace('  ', ' ')
    final_text = final_text.strip()
    
    return final_text

def pad_sequence(tok_seq, pad_tok, max_len=1000):
    cur_len = len(tok_seq)
    if cur_len >= max_len:
        return tok_seq[:max_len]

    padding = [pad_tok]*(max_len - cur_len)
    text = tok_seq + padding
    
    return text

def build_vocab(df, tokenizer):
    counter = Counter()
    df['full_text'].apply(lambda string_ : counter.update(tokenizer(string_)))
    
    return vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])

vocab_ = build_vocab(train_df, tokenizer)
vocab_.set_default_index(vocab_['<unk>'])

def data_process(df):
    data = []
    df['full_text_'] = df['full_text'].apply(lambda text : mark_bos_eos(text))
    for raw_en, label in zip(df['full_text_'], df[['cohesion', 'syntax', 'vocabulary', 'phraseology', 'grammar', 'conventions']].values.tolist()):
        tokens = [vocab_[token] for token in tokenizer(raw_en)]
        token_seq = pad_sequence(tokens, vocab_['<pad>'], MAX_SENT_LEN)
        data.append((token_seq, label))
        
    return data

train_data = data_process(train_df)
val_data = data_process(val_df)
test_data = data_process(test_df)

In [135]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

BATCH_SIZE = 128
PAD_IDX = vocab_['<pad>']
BOS_IDX = vocab_['<bos>']
EOS_IDX = vocab_['<eos>']

def generate_batch(data_batch):
    en_batch = []
    label_batch = []
    
    for (en_item, label_item) in data_batch:
        en_batch.append(en_item)
        label_batch.append(label_item)
    
    en_batch = torch.tensor(en_batch, dtype=torch.long)
    label_batch = torch.tensor(label_batch, dtype=torch.float)
    
    return en_batch, label_batch

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE, shuffle=False, collate_fn=generate_batch)


cpu


In [136]:
class MODEL(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 enc_hid_dim: int,
                 out_dim: int,
                 dropout: float):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.enc_hid_dim = enc_hid_dim
        self.out_dim = out_dim
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.lstm = nn.LSTM(emb_dim, enc_hid_dim, num_layers=1, bidirectional=False, batch_first=True, dropout=dropout)
        
        self.fc = nn.Linear(MAX_SENT_LEN * enc_hid_dim, out_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src: Tensor) -> Tuple[Tensor]:
        
        embedded = self.dropout(self.embedding(src))
        
        outputs, _ = self.lstm(embedded)
        
        outputs = torch.reshape(outputs, (outputs.shape[0], outputs.shape[1]*outputs.shape[2]))
        
        fc_out = self.fc(outputs)
        
        return fc_out
    

In [137]:
INPUT_DIM = len(vocab_)
EMB_DIM = 256
GRU_HID_DIM = 128
OUTPUT_DIM = 6
DROPOUT = 0.2


In [138]:
model = MODEL(INPUT_DIM, EMB_DIM, GRU_HID_DIM, OUTPUT_DIM, DROPOUT)
model.to(device)

def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)
            

model.apply(init_weights)

MODEL(
  (embedding): Embedding(24777, 256)
  (lstm): LSTM(256, 128, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=128000, out_features=6, bias=True)
  (dropout): Dropout(p=0.2, inplace=False)
)

In [139]:
class MCRMSE(nn.Module):
    def __init__(self, output_dim):
        super().__init__()
        self.mse = nn.MSELoss(reduction='sum')
        self.no_cols = output_dim
        
    def forward(self, target, output):
        rmse = torch.pow(torch.div(self.mse(target, output), output.shape[0]), 0.5)
        mcrmse = torch.div(rmse, self.no_cols)
        
        return mcrmse

In [140]:
criterion = MCRMSE(OUTPUT_DIM)

optimizer = optim.Adam(model.parameters())

def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 7,308,550 trainable parameters


In [141]:
def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):
    
    model.train()
    
    epoch_loss = 0
    
    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)
        
        optimizer.zero_grad()
        
        output = model(src)
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
        
        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            
            output = model(src)
            
            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
    
    return epoch_loss / len(iterator)

def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} | Val. PPL: {math.exp(valid_loss):7.3f}')
    
    

Epoch: 01 | Time: 1m 35s
	Train Loss: 0.496 | Train PPL:   1.642
	 Val. Loss: 0.314 | Val. PPL:   1.369
Epoch: 02 | Time: 1m 35s
	Train Loss: 0.264 | Train PPL:   1.302
	 Val. Loss: 0.223 | Val. PPL:   1.250
Epoch: 03 | Time: 1m 35s
	Train Loss: 0.216 | Train PPL:   1.241
	 Val. Loss: 0.236 | Val. PPL:   1.267
Epoch: 04 | Time: 1m 35s
	Train Loss: 0.201 | Train PPL:   1.223
	 Val. Loss: 0.220 | Val. PPL:   1.246
Epoch: 05 | Time: 1m 30s
	Train Loss: 0.182 | Train PPL:   1.199
	 Val. Loss: 0.221 | Val. PPL:   1.248
Epoch: 06 | Time: 1m 30s
	Train Loss: 0.173 | Train PPL:   1.188
	 Val. Loss: 0.238 | Val. PPL:   1.269
Epoch: 07 | Time: 1m 34s
	Train Loss: 0.166 | Train PPL:   1.180
	 Val. Loss: 0.232 | Val. PPL:   1.262
Epoch: 08 | Time: 1m 29s
	Train Loss: 0.152 | Train PPL:   1.164
	 Val. Loss: 0.236 | Val. PPL:   1.266
Epoch: 09 | Time: 1m 34s
	Train Loss: 0.149 | Train PPL:   1.161
	 Val. Loss: 0.240 | Val. PPL:   1.271
Epoch: 10 | Time: 1m 33s
	Train Loss: 0.136 | Train PPL:   1.146

In [143]:
test_output = np.empty((0,6), float)

with torch.no_grad():
    for _, (src, trg) in enumerate(test_iter):
        src, trg = src.to(device), trg.to(device)
        pred = model(src)
        pred = pred.to('cpu')
        test_output = np.append(test_output, pred, axis=0)
        
test_output = test_output.T

In [144]:
test_output.shape

(6, 3)

In [145]:
submission_df = pd.DataFrame({'text_id':test_df['text_id'], 'cohesion':test_output[0],
                             'syntax': test_output[1], 'vocabular': test_output[2],
                             'phraseology': test_output[3], 'grammar': test_output[4],
                             'conventions': test_output[5]})

In [146]:
submission_df.to_csv('sample_submission.csv', index=False)